# Outage probability versus $\gamma_{th}$

## Loading libraries

In [1]:
import numpy as np

from misc import single_reflection_coordinate
from misc import reflection_coordinates
from misc import reflection_angles
from misc import line_equation_coefficients
from misc import dist_to_obstructions
from misc import blocked_rays_main_paths
from misc import fcombine

## Simulation settings

In [2]:
# np.random.seed(1234)

trials = 10000

GHz  = 18 # operating frequency
c    = 299792458  # speed of light
r    = 0.1  # obtruction radius
lmbd = 0.5  # intensity of the PPP (m^(-2))
sgm2 = 1.658e-11
Pt   = 0.1

# Table 2 (paper)
if (GHz == 18):
    z  = 130
    AO = 0.00006
    f  = 18e9
if (GHz == 26):
    z  = 200
    AO = 0.00013
    f  = 26e9
if (GHz == 60):
    z  = 390
    AO = 0.015
    f  = 60e9
if (GHz == 73):
    z  = 420
    AO = 0.0075
    f  = 73e9

wvlen= c / f
# Refractive indices
n1          = 1
n2          = 1.5

# Antenna gain
GtdB = 20 # Gain at the transmitter in dB   
GrdB = 10 # Gain at the receiver in dB
Gt   = np.power(10, GtdB/10)
Gr   = np.power(10, GrdB/10)
G    = Gr * Gt
B    = sgm2/(Pt * G)

## Room, RIS, Tx, and Rx

In [3]:
# Room dimensions
Lx = 30
Ly = 20
room_area = Lx * Ly

ris_center_x = 7.5
ris_center_y = 0
RIS_antenna = 'Intelligent array'
N = 64
# RIS on the bottom wall
if (ris_center_y == 0):
    x_RIS  = wvlen * (np.linspace(-N/2,(N/2)-1,N) - 0.50) + ris_center_x
    y_RIS  = np.zeros(N) 
    wall = 'bottom'
# RIS on the left wall
if (ris_center_x == 0):
    y_RIS  = wvlen * (np.linspace(-N/2,(N/2)-1,N) - 0.50) + ris_center_y
    x_RIS  = np.zeros(N)
    wall = 'left'
# RIS on the upper wall
if (ris_center_y == Ly):
    x_RIS  = wvlen * (np.linspace(-N/2,(N/2)-1,N) - 0.50) + ris_center_x    # RIS x-coordinates
    y_RIS  = np.ones(N) * Ly
    wall = 'upper'
# RIS on the right wall
if (ris_center_x == Lx):
    y_RIS  = wvlen * (np.linspace(-N/2,(N/2)-1,N) - 0.50) + ris_center_y     # RIS x-coordinates
    x_RIS  = np.ones(N) * Lx
    wall = 'right'


# Transmitter and receiver cooordinates
x_Tx = 1
y_Tx = 1

x_Rx = 1 + 10/np.sqrt(2)
y_Rx = 1 + 10/np.sqrt(2)

## Computing free path loss

In [4]:
# Reflection path coordinates
xr, yr = reflection_coordinates(Lx, Ly, x_Tx, y_Tx, x_Rx, y_Rx)
# Reflection angles
theta_i = reflection_angles(x_Tx, y_Tx, x_Rx, y_Rx, xr, yr)

# LOS path (coefficients of line equations)
TxRx = line_equation_coefficients(x_Tx, x_Rx, y_Tx, y_Rx, 'single')
# Tx-Reflection paths (coefficients of line equations)
TxRP = line_equation_coefficients(x_Tx, xr, y_Tx, yr, 'multiple')
# Reflection paths-Rx (coefficients of line equations)
RPRx = line_equation_coefficients(xr, x_Rx, yr, y_Rx, 'multiple')
# Tx-RIS paths (coefficients of line equations)
TxRIS = line_equation_coefficients(x_Tx, x_RIS, y_Tx, y_RIS, 'multiple')
# RIS-Rx paths (coefficients of line equations)
RISRx = line_equation_coefficients(x_RIS, x_Rx, y_RIS, y_Rx, 'multiple')
# Tx-RIScenter 
TxRISc = line_equation_coefficients(x_Tx, ris_center_x, y_Tx, ris_center_y, 'single')
# Tx-RIScenter 
RIScRx = line_equation_coefficients(ris_center_x, x_Rx, ris_center_y, y_Rx, 'single')

dk      = np.zeros(5)
dk[0]   = np.sqrt(np.power(x_Rx - x_Tx, 2) + np.power(y_Rx - y_Tx, 2))
dk[1:5] = TxRP.d + RPRx.d
dRIS    = TxRIS.d + RISRx.d
dRISc   = TxRISc.d + RIScRx.d

Rs       = np.divide(n1 * np.cos(theta_i) - n2 * np.sqrt(1 - np.power(n1 * np.sin(theta_i) / n2, 2)),
                     n1 * np.cos(theta_i) + n2 * np.sqrt(1 - np.power(n1 * np.sin(theta_i) / n2, 2)))
Rp       = np.divide(n1 * np.sqrt(1 - np.power(n1 * np.sin(theta_i) / n2, 2)) - n2 * np.cos(theta_i),
                     n1 * np.sqrt(1 - np.power(n1 * np.sin(theta_i) / n2, 2)) + n2 * np.cos(theta_i))
Rs       = Rs * np.conj(Rs)
Rp       = Rp * np.conj(Rp)
Rk       = np.divide(2, (Rs+Rp))
Wk       = np.zeros(5)
Lk       = np.zeros(5)
Wk[1:5]  = 10 * np.log10(Rk)

# Free space path loss for LOS path and reflection paths
Lk      = 20 * np.log10(4 * np.pi * f * dk / c) + AO * dk + Wk[0:5]  # Eq. (1)

theta_id = 180/np.pi * theta_i
print('coordinates')
print("Tx:\t (%.2f, %.2f)"%(x_Tx, y_Tx))
print("Rx:\t (%.2f, %.2f)"%(x_Rx, y_Rx))
print('--------------------------------')
print('\t\t LOS\t\t path1\t\t path2\t\t path3\t\t path4')
print('distances[m]\t %.5f\t %.5f\t %.5f\t %.5f\t %.5f'%(dk[0], dk[1], dk[2], dk[3], dk[4]))
print('angles[deg]\t %.5f\t %.5f\t %.5f\t %.5f\t %.5f'%(0, theta_id[0], theta_id[1], theta_id[2], theta_id[3]))
print('W[dB]\t\t %.5f\t %.5f\t %.5f\t %.5f\t %.5f'%(Wk[0], Wk[1], Wk[2], Wk[3], Wk[4]))
print('Lk[dB]\t\t %.5f\t %.5f\t %.5f\t %.5f\t %.5f'%(Lk[0], Lk[1], Lk[2], Lk[3], Lk[4]))
print('--------------------------------')

coordinates
Tx:	 (1.00, 1.00)
Rx:	 (8.07, 8.07)
--------------------------------
		 LOS		 path1		 path2		 path3		 path4
distances[m]	 10.00000	 11.50149	 51.41747	 31.72694	 11.50149
angles[deg]	 0.00000	 37.93708	 7.90452	 12.87781	 37.93708
W[dB]		 0.00000	 13.52086	 13.97875	 13.97471	 13.52086
Lk[dB]		 77.55383	 92.28987	 105.75728	 101.55841	 92.28987
--------------------------------
